In [1]:
# https://www.youtube.com/watch?v=aQaZMC9-Jok

import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential  # Can not be moved
from tensorflow.keras.models import load_model
from keras.layers import Conv2D, MaxPooling2D  # Can not be moved
from keras.layers import Activation, Dropout, Flatten, Dense # Can not be moved
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join

from keras import optimizers

import matplotlib.pyplot as plt

from zipfile import ZipFile
import boto3

Using TensorFlow backend.


In [ ]:
with ZipFile("dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")


In [ ]:
import shutil
shutil.rmtree('train20')

In [ ]:
bucket='tog-dataset-sagemaker' # Replace with your s3 bucket name
Filename = "dataset.zip"
s3 = boto3.Session().resource('s3').Bucket(bucket).Object(Filename).download_file('dataset.zip')

In [ ]:
def imshow(image_RGB):
    image_0_2_1 = image_RGB
    print(image_0_2_1.shape)
    plt.imshow(image_0_2_1)


In [ ]:
img_path = "data/data/train/cat/cat.3.jpg"
img = load_img(img_path, target_size=(100, 100))

imgArray = img_to_array(img)
# Update to 0-1 format
imgArray = imgArray / 255.0
print(imgArray.shape)
plt.imshow(imgArray)


In [ ]:
train_data = "data/data/train"
test_data = "data/data/test"

# Número de iteraciones sobre todo el dataset de entrenamiento
epochs = 20

# Dimensiones de las imágenes para procesar
n_H, n_W = 100, 100

# Utilizaremos mini-batch
batch_size = 32

# Número de iteraciones que vamos a procesar la información en cada epoca (entrenamiento)
train_samples = 1000

# Número de iteraciones que vamos a procesar la información en cada epoca (validación)
validation_samples = 200

# Definamos la tasa de aprendizaje
learning_rate = 0.05

# Número de clases
class_num = 3

# Estructura de la red neuronal convolucionales
filter_conv1 = 32
size_filter1 = (3,3)

filter_conv2 = 64
size_filter2 = (2,2)

#Usaremos un Max Pooling
size_pool = (2,2)


# 1. Antes de comenzar con el modelo, vamos a preprocesar las imágenes
train_datagen = ImageDataGenerator(
    rescale = 1./255,          # Normalizar los valores de los pixeles
    shear_range= 0.3,           # Rango del ángulo que podemos inclinar nuestras imágenes
    zoom_range = 0.3,            # Rango del zoom que podemos hacer a nuestras imágenes
    horizontal_flip = True       # Invierte imágenes
)

test_datagen = ImageDataGenerator(
    rescale = 1./255           # Normalizar los valores de los pixeles
)

# Accede al directorio, preprocesa las imágenes y organiza en mini-batchs
train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size = (n_H, n_W),             # Tamaño de las imágenes
    batch_size = batch_size,              # Tamaño del mini-batch
    class_mode = 'categorical'            # Modelo para clasificación
)

# Accede al directorio, preprocesa las imágenes y organiza en mini-batchs
validation_generator = test_datagen.flow_from_directory(
    test_data,
    target_size = (n_H, n_W),
    batch_size = batch_size,
    class_mode = 'categorical'
)


In [ ]:
# Check for TensorFlow or Thieno
if K.image_data_format() == "channels_first":
    input_shape = (3, n_W, n_H)
else:
    input_shape = (n_W, n_H, 3)



In [ ]:
# 2. Crear la ConvNet
def createModel():
    model = Sequential()
    model.add(Conv2D(filter_conv1, size_filter1, padding="same", input_shape=(n_H, n_W,3), activation="relu"))
    model.add(MaxPooling2D(pool_size=size_pool))
    model.add(Conv2D(filter_conv2, size_filter2, padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=size_pool))
    model.add(Flatten())
    model.add(Dense(255, activation=None))
    model.add(Dropout(0.5))
    model.add(Dense(class_num, activation='softmax'))

    return model

In [ ]:
print(train_generator.class_indices)
imgs, labels = next(train_generator)

In [ ]:
image_batch, label_batch = train_generator.next()

print(len(image_batch))
for index in range(0, len(image_batch)):
    image = image_batch[index]
    print(label_batch[index])
    imshow(image)

In [ ]:
model = createModel()
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(learning_rate),  
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=train_samples, 
    epochs=epochs, 
    validation_data = validation_generator, 
    validation_steps=validation_samples)

In [ ]:
model.save("model.h5")
model.save_weights("weights.h5")


In [ ]:
bucket='sn2020-final-project' # Replace with your s3 bucket name
keymodel = 'model.h5'
keyweights = "weights.h5"

url = 's3://{}/{}'.format(bucket, keymodel)
boto3.Session().resource('s3').Bucket(bucket).Object(keymodel).upload_file('model.h5')
print('Done writing to {}'.format(url))

url = 's3://{}/{}'.format(bucket, keyweights)
boto3.Session().resource('s3').Bucket(bucket).Object(keyweights).upload_file('weights.h5')
print('Done writing to {}'.format(url))


In [2]:
bucket='sn2020-final-project' # Replace with your s3 bucket name
file = 'DenseNet MLT.ipynp'


url = 's3://{}/{}'.format(bucket, file)
boto3.Session().resource('s3').Bucket(bucket).Object(file).upload_file(file)
print('Done writing to {}'.format(url))


FileNotFoundError: [Errno 2] No such file or directory: 'utils.ipynp'

In [ ]:
# Lets plot the results
print(history.history.keys())
# history accuracy
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.title("model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
# history accuracy
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
s3 = boto3.Session().resource('s3').Bucket(bucket).Object(keyweights).download_file('weights.h5')

In [ ]:
# Evaluate
model = createModel()
model.load_weights("weights.h5")

img_path = test_data + os.sep + "gorilla" + os.sep + "18.jpeg"
img = load_img(img_path, target_size=(n_H, n_W))

imgArray = img_to_array(img)
# Update to 0-1 format
imgArray = imgArray / 255.0
print(imgArray.shape)
plt.imshow(imgArray)
# The modelexspect batches then lets do a batch of 1 
imgArray = np.expand_dims(imgArray, axis=0)

# Predict
classList = os.listdir(test_data)
prediction = model.predict(imgArray)
print(prediction)
prediction = model.predict_classes(imgArray)
print(classList[prediction[0]])




In [ ]:
# Lets predict the gorillas
gorillapath = test_data + os.sep + "cat" + os.sep
imagesPredict = [f for f in listdir(gorillapath) if isfile(join(gorillapath,f))]
# print(imagesPredict)

for file in imagesPredict:
    img = load_img(gorillapath + file, target_size=( n_W, n_H))
    tensorImage = img_to_array(img) / 255.
    tensorImage = np.expand_dims(tensorImage, axis=0)
    gorillaPrediction = model.predict_classes(tensorImage, batch_size = 1)
    print("file "+ file + " is: " + classList[gorillaPrediction[0]])

        